In [74]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader as pdr
import pandas._testing as tm
import numpy as np
import math

In [75]:
data = yf.download("BTC-USD, GC=F, ^GSPC, ^IXIC, ^VIX", start="2022-01-02", end='2022-04-26')


[*********************100%***********************]  5 of 5 completed


In [76]:
df = data[['Volume', "Close"]] # Оставляем данные: Объем и дневная Цена закрытия
df1 = df.drop([('Volume','^VIX')], axis=1) # Убираем колонку "Объем" "Волатильность" т.к. отсутствуют данные

In [112]:
data[["Close"]]

Close                                                   
                 BTC-USD         GC=F        ^GSPC         ^IXIC       ^VIX
Date                                                                       
2022-01-01  47686.812500          NaN          NaN           NaN        NaN
2022-01-02  47345.218750          NaN          NaN           NaN        NaN
2022-01-03  46458.117188  1799.400024  4796.560059  15832.799805  16.600000
2022-01-04  45897.574219  1814.000000  4793.540039  15622.719727  16.910000
2022-01-05  43569.003906  1824.599976  4700.580078  15100.169922  19.730000
...                  ...          ...          ...           ...        ...
2022-04-21  40527.363281  1944.900024  4393.660156  13174.650391  22.680000
2022-04-22  39740.320312  1931.000000  4271.779785  12839.290039  28.209999
2022-04-23  39486.730469          NaN          NaN           NaN        NaN
2022-04-24  39469.292969          NaN          NaN           NaN        NaN
2022-04-25  40458.308594  1893.199951  4296.120117  13004.849609  27.020000

[115 rows x 5 columns]

In [118]:
df = data[([('Close', '^GSPC'), ('Close', 'BTC-USD')])]
df.columns = '^GSPC','BTC-USD' #
ind = df.index
# df1 = df.fillna(0, axis=0)
df

,^GSPC,BTC-USD
Date,,
2022-01-01,NaN,47686.812500
2022-01-02,NaN,47345.218750
2022-01-03,4796.560059,46458.117188
2022-01-04,4793.540039,45897.574219
2022-01-05,4700.580078,43569.003906
...,...,...
2022-04-21,4393.660156,40527.363281
2022-04-22,4271.779785,39740.320312
2022-04-23,NaN,39486.730469


In [119]:
# Подставим средние значения в неторговые дни по золоту. 
# Затем нужно убрать данную подстановку и посмотреть на изменение корреляции.
df1 = df.fillna(df.mean())
df1

,^GSPC,BTC-USD
Date,,
2022-01-01,4459.056015,47686.812500
2022-01-02,4459.056015,47345.218750
2022-01-03,4796.560059,46458.117188
2022-01-04,4793.540039,45897.574219
2022-01-05,4700.580078,43569.003906
...,...,...
2022-04-21,4393.660156,40527.363281
2022-04-22,4271.779785,39740.320312
2022-04-23,4459.056015,39486.730469


In [101]:
df1.loc[df1['^GSPC']==0, ['BTC-USD']]=0 #Для тренировки
df1 = df1[df1['^GSPC']>0]
df1

,^GSPC,BTC-USD
Date,,
2022-01-03,4796.560059,46458.117188
2022-01-04,4793.540039,45897.574219
2022-01-05,4700.580078,43569.003906
2022-01-06,4696.049805,43160.929688
2022-01-07,4677.029785,41557.902344
...,...,...
2022-04-19,4462.209961,41502.750000
2022-04-20,4459.450195,41374.378906
2022-04-21,4393.660156,40527.363281


In [102]:
# В публикациях стала чаще упоминаться гипотеза, что ранее не было зависимости Биткоина от биржевых индексов, 
# но с течением времени эта корреляция становиться более заметной. Проверим данную гипотезу.
# Произведем расчет коэффициэнта корреляции с начала 2022 года.

x = df1['^GSPC']
y = df1['BTC-USD'] 
cor_ch = (len(df1)*sum(x*y))-(sum(x)*sum(y))
cor_zn = (math.sqrt(((len(df1)*(sum(x**2) - (sum(x))**2)))*((len(df1)*(sum(y**2) - (sum(y))**2)))))
#empt = pd.DataFrame({('r'): 0}, index=ind)
#df2 = pd.concat([df1, empt], axis=1)    
cor_ch
cor_zn
n = (cor_ch/cor_zn)
n

1.674037711418278e-05

In [103]:
import plotly.express as px
fig = px.scatter(x=x,y=y)
fig.show()

